In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/train-nlp/train (1).csv
/kaggle/input/test-nlp/test (1).csv


In [ ]:
import numpy as np
import pandas as pd
import torch

In [ ]:
df = pd.read_csv('/kaggle/input/train-nlp/train (1).csv')
df_test = pd.read_csv('/kaggle/input/test-nlp/test (1).csv')

In [ ]:
!pip install googletrans
!pip install --upgrade googletrans==4.0.0-rc1

##Dataset Augmentation

In [ ]:
'''for data augmentation, using back translation. Converts into french first and then back to english'''

from googletrans import Translator

def back_translate(sentence, target_language='fr'):
    translator = Translator()
    translated = translator.translate(sentence, dest=target_language)
    back_translated = translator.translate(translated.text, dest='en')
    return back_translated.text


Original: Hello, my name is Ashmit and I like ice-cream.
Back-translated: Hello, My name is Ashmit and I like ice.


In [ ]:
new_samples = []


for i in tqdm(range(len(df))):
    original_text = df['Text'][i]
    original_emotion = df['Emotion'][i]


    translated_text = back_translate(original_text)

    new_sample = {'Text': translated_text, 'Emotion': original_emotion}

    new_samples.append(new_sample)

new_df = pd.DataFrame(new_samples)

df = pd.concat([df, new_df], ignore_index=True)

100%|██████████| 240/240 [00:40<00:00,  5.92it/s]


In [ ]:
df = df.sample(frac=1.0, random_state=43).reset_index(drop=True)

##Another library I found for dataset augmentation. Basically generates n sentences for the input sentence. the outputs are not coherent sentences but similar sentences having words with wrong spellings.

In [ ]:
!pip install nlpaug

In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas

In [ ]:
aug = nac.KeyboardAug()
augmented_text = aug.augment(text, n=2)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

In [ ]:
import pandas as pd
from tqdm import tqdm

new_samples = []

for i in tqdm(range(len(df))):
    original_text = df['Text'][i]
    original_emotion = df['Emotion'][i]

    for _ in range(10):
        translated_text = aug.augment(original_text)


        if isinstance(translated_text, list):
            translated_text = translated_text[0]

        new_sample = {'Text': translated_text, 'Emotion': original_emotion}

        new_samples.append(new_sample)

new_df = pd.DataFrame(new_samples)

df = pd.concat([df, new_df], ignore_index=True)

In [ ]:
df

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
!pip install pytesseract transformers==4.28.1 datasets evaluate rouge-score nltk tensorboard py7zr

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import glob
from datasets import load_dataset
import datasets

#Flan T5 - Gave 70 on public leaderboard. lr = 2e-4, epochs 2, dataset size 480 rows

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
model = TFAutoModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
df.drop('Emotion', axis = 1, inplace = True)

In [ ]:
input = tokenizer('Hello world')
input

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
df_test = pd.read_csv('/kaggle/input/test-nlp/test (1).csv')

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
from datasets import Dataset

In [ ]:
import pandas as pd
from datasets import Dataset


'''label_to_text = {
    0: 'sadness',
    1: 'anger',
    2: 'joy',
    3: 'love',
    4: 'surprise',
    5: 'fear'
}'''

#converting to dict
data_dict = {
    "text": df['Text'].tolist(),
    "label": df['Emotion'].tolist()
}

test_dict = {
    "text": df_test['Text'].tolist()
}


dataset = Dataset.from_dict(data_dict)
testset = Dataset.from_dict(test_dict)

print(dataset[49])
print(testset[209])


{'text': 'I train every morning before and I feel fabulous for that', 'label': 'joy'}
{'text': 'i feel kinda strange too cause i didnt encountered with such feelings last year'}


In [ ]:
df

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = dataset.map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = dataset.map(lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

  0%|          | 0/1 [00:00<?, ?ba/s]

Max source length: 53


  0%|          | 0/1 [00:00<?, ?ba/s]

Max target length: 2


In [ ]:
def preprocess_function(sample, padding="max_length"):
    # adding prefix to the input for t5
    inputs = [item for item in sample["text"]]

    # tokenizing inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenizing targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, we replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text', 'label'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset.features)}")

  0%|          | 0/1 [00:00<?, ?ba/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [ ]:
!pip install transformers

In [ ]:
pip install -U accelerate

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

metric = evaluate.load("f1")

#preprocessing
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, average='macro')
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:


from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

repository_id = f"{model_id.split('/')[1]}-imdb-text-classification"

# training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=2e-4,

    num_train_epochs=2,
    evaluation_strategy="no",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    report_to="tensorboard",
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset
)

In [ ]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
60,0.317400
120,0.018200


TrainOutput(global_step=120, training_loss=0.16778303682804108, metrics={'train_runtime': 44.7407, 'train_samples_per_second': 21.457, 'train_steps_per_second': 2.682, 'total_flos': 71899522007040.0, 'train_loss': 0.16778303682804108, 'epoch': 2.0})

In [ ]:
from tqdm.auto import tqdm

samples_number = len(testset)
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
    text = testset['text'][i]
    inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
    outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions_list.append(prediction)
    #labels_list.append(dataset['test']['label'][i])

    progress_bar.update(1)

  0%|          | 0/210 [00:00<?, ?it/s]

In [ ]:
predictions_list = pd.DataFrame(predictions_list)

In [ ]:
df_1 = pd.concat([df_test, predictions_list], ignore_index=True, axis = 1)

In [ ]:
df_1 = df_1.rename(columns={0: 'ID', 1: 'Text', 2: 'Emotion'})

In [ ]:
df_1.drop('Text', axis = 1, inplace = True)

In [ ]:
df_1

,ID,Emotion
0,0,joy
1,1,anger
2,2,sadness
3,3,love
4,4,joy
...,...,...
205,205,surprise
206,206,surprise
207,207,surprise
208,208,sadness


In [ ]:
df_1.to_csv('predicted_results_23_nlp.csv', index=False)

#Bert gave 52 on public leader board. lr 1e-5, 120, 6 epochs

In [ ]:
model = TFAutoModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
import pandas as pd
from datasets import Dataset

#mapping
label_to_text = {
    0: 'sadness',
    1: 'anger',
    2: 'joy',
    3: 'love',
    4: 'surprise',
    5: 'fear'
}

df['label_text'] = df['Emotion'].map({v: k for k, v in label_to_text.items()})

# converting to a dictionary format.
data_dict = {
    "text": df['Text'].tolist(),
    "label": df['label_text'].tolist(),
    "label_text": df['Emotion'].tolist()
}

test_dict = {
    "text": df_test['Text'].tolist()
}


dataset = Dataset.from_dict(data_dict)
testset = Dataset.from_dict(test_dict)

print(dataset[49])
print(testset[209])


In [ ]:
emotions_encoded = dataset.map(tokenize, batched=True, batch_size=None)
emotions_encoded_test = testset.map(tokenize, batched=True, batch_size=None)

In [ ]:
emotions_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

BATCH_SIZE = 32

def order(inp):
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

train_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded[:])
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded_test[:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [ ]:
classifier = BERTForClassification(model, num_classes=6)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = classifier.fit(
    train_dataset,
    epochs=3
)

In [ ]:
predictions = classifier.predict(test_dataset)

In [ ]:
label_to_text = {
    0: 'sadness',
    1: 'anger',
    2: 'joy',
    3: 'love',
    4: 'surprise',
    5: 'fear'
}

predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
predicted_labels

In [ ]:
predicted_label_texts = [label_to_text[label] for label in predicted_labels]

df_test['Emotion'] = predicted_label_texts

In [ ]:
df_test.drop('Text', axis=1, inplace = True)

In [ ]:
df_test.to_csv('predicted_results_17_nlp.csv', index=False)

In [ ]:
pip install torch==2.0.1

# Setfit gave 43 on public leaderboard. Didnt get time to finetune and check for different hps.

In [ ]:
!pip install git+https://github.com/pmbaumgartner/setfit -q

In [ ]:
from setfit import SetFitClassifier

In [ ]:
clf = SetFitClassifier("paraphrase-MiniLM-L3-v2")

In [ ]:
train_form = df

In [ ]:
train_form['Emotion_label'] = pd.DataFrame(train_form['Emotion'].replace('fear',0).replace('surprise',1).replace('sadness',2).replace('joy',3).replace('anger',4).replace('love',5))

In [ ]:
docs = train_form['Text'].to_list()
labels = train_form['Emotion'].to_list()

In [ ]:
clf.fit(docs, labels)

In [ ]:
df_test = pd.read_csv('/kaggle/input/train-nlp/test (1).csv')

In [ ]:
df_test['Emotion'] = clf.predict(df_test['Text'].to_list())

In [ ]:
df_test.drop('Text', axis = 1, inplace = True)

In [ ]:
df_test.to_csv('predicted_results_18_nlp.csv', index=False)